In [34]:
import numpy as np
from PIL import Image

In [35]:
folder = 'origin'
fname = '002_001'

In [36]:
pic = Image.open(folder + '\\' + fname + '.jpg')

In [37]:
image = np.array(pic)

In [38]:
# create boolean mask, white is True
white = np.array([255, 255, 255])

In [39]:
def get_mask(image):
    return np.abs(image - white).sum(axis=2) < 0.05

In [40]:
# o = orientation, 0 for horizontal, 1 for vertical
def clip_pos(mask, orientation):
    not_all_white = np.any(~mask, axis=orientation)
    all_white = np.where(~not_all_white)
    return all_white[0][0]

In [41]:
# trim white borders
mask0 = get_mask(image)

coords = np.array(np.nonzero(~mask0))
top_left = np.min(coords, axis=1)
bottom_right = np.max(coords, axis=1)

trim = 10

out = image[top_left[0]+trim:bottom_right[0]-trim,
            top_left[1]+trim:bottom_right[1]-trim]

In [42]:
# create img1 (full image, all white space removed)
mask1 = get_mask(out)
clip1 = clip_pos(mask1, 1)
main_image = out[:clip1, :]

In [43]:
# trim up scale
clipped1 = out[clip1:, :]
mask2 = get_mask(clipped1)
clip2 = clip_pos(mask2, 0)
clipped2 = clipped1[:, clip2:]
mask3 = get_mask(clipped2)
any_colored = np.where(np.any(~mask3, axis=0))[0][0]
clipped3 = clipped2[:, any_colored:]
mask4 = get_mask(clipped3)
clip4 = clip_pos(mask4, 0)
clipped4 = clipped3[:, :clip4]

In [44]:
# main image
img1 = Image.fromarray(main_image)
img1.putalpha(255)

# scale image
img2 = Image.fromarray(clipped4)
img2.putalpha(255)

In [45]:
# make scale transparent
datas = img2.getdata()
newData = []
for item in datas:
    if item[0] > 200 and item[1] > 100 and item[2] > 100:
        newData.append((255, 255, 255, 0))
    else:
        newData.append((0, 0, 0, 255))
        
img2.putdata(newData)

In [46]:
# paste and save, tweak to modify scale location
img1.paste(img2, (200, 1350), img2)

In [47]:
img1 = img1.convert("RGB")

In [48]:
img1 = img1.save('output//with_scale.jpg')